In [8]:
from pathlib import Path

from cadet_simplified import (
    list_column_models,
    list_binding_models,
    list_operation_modes,
    get_operation_mode,
    ExcelTemplateGenerator,
    parse_excel,
    plotting,
    SimulationRunner,
    FileStorage,
)

In [9]:
print(100 * "=")
print("SUPPORTED COLUMN MODELS")
print(list_column_models())
print(100 * "-")
print("SUPPORTED BINDING MODELS")
print(list_binding_models())
print(100 * "-")
print("SUPPORTED OPERATION MODES")
print(list_operation_modes())
print(100 * "=")

SUPPORTED COLUMN MODELS
['GeneralRateModel', 'LumpedRateModelWithPores', 'LumpedRateModelWithoutPores']
----------------------------------------------------------------------------------------------------
SUPPORTED BINDING MODELS
['BiLangmuir', 'GeneralRateModel', 'GeneralizedIonExchange', 'Langmuir', 'Linear', 'LumpedRateModelWithPores', 'LumpedRateModelWithoutPores', 'StericMassAction']
----------------------------------------------------------------------------------------------------
SUPPORTED OPERATION MODES
['LWE_concentration_based']


In [11]:
col = "LumpedRateModelWithPores"
bim = "StericMassAction"
opm = "LWE_concentration_based"
n_components = 3

template_generator = ExcelTemplateGenerator(
    operation_mode=opm,
    column_model=col,
    binding_model=bim,
    n_components=n_components,
    component_names=["Salt", "Product", "Impurity1"],
)

save_path = Path("~").expanduser()
excel_template_file = save_path / "test_template.xlsx"
template_generator.save(excel_template_file)
print(f"Template saved to: {excel_template_file}")


Template saved to: /home/moritz/test_template.xlsx


In [12]:
filled_template_path = save_path / "test_template_filled.xlsx"


result = parse_excel(filled_template_path)


operation_mode = get_operation_mode(opm)
process_list = []

for exp in result.experiments:
    process = operation_mode.create_process(exp, result.column_binding)
    process_list.append(process)
    

    print(process.check_config())


runner = SimulationRunner()

print("\nRunning simulations...")
results = runner.run_batch(
    process_list,
    stop_on_error=False,
    progress_callback=lambda current, total, res: print(
        f"  [{current}/{total}] {res.experiment_name}: "
        f"{'Success' if res.success else 'Failed'} ({res.runtime_seconds:.2f}s)"
    ),
)


storage_dir = save_path / "cadet_experiments"
storage = FileStorage(storage_dir)

set_id = storage.save_experiment_set(
    name="my_study",
    operation_mode=opm,
    experiments=result.experiments,
    column_binding=result.column_binding,
    results=results,
    description="Test simulation run",
)


True
True
True

Running simulations...
  [1/3] experiment_3: Success (4.39s)
  [2/3] experiment_2: Success (4.58s)
  [3/3] experiment_1: Success (5.09s)


In [5]:
df = storage.list_experiments(limit=10)
df

,experiment_set_id,experiment_set_name,experiment_name,created_at,n_components,component_names,operation_mode,column_model,binding_model,has_results,has_chromatogram
0,ed30db1ce36a,my_study,experiment_1,2026-01-28T21:58:16.105006,3,"Salt, Product, Impurity1",LWE_concentration_based,LumpedRateModelWithPores,StericMassAction,True,True
1,ed30db1ce36a,my_study,experiment_2,2026-01-28T21:58:16.105006,3,"Salt, Product, Impurity1",LWE_concentration_based,LumpedRateModelWithPores,StericMassAction,True,True
2,ed30db1ce36a,my_study,experiment_3,2026-01-28T21:58:16.105006,3,"Salt, Product, Impurity1",LWE_concentration_based,LumpedRateModelWithPores,StericMassAction,True,True
3,9cd4e7392512,my_study,experiment_1,2026-01-28T21:56:37.916777,3,"Salt, Product, Impurity1",LWE_concentration_based,LumpedRateModelWithPores,StericMassAction,True,True
4,9cd4e7392512,my_study,experiment_2,2026-01-28T21:56:37.916777,3,"Salt, Product, Impurity1",LWE_concentration_based,LumpedRateModelWithPores,StericMassAction,True,True
5,9cd4e7392512,my_study,experiment_3,2026-01-28T21:56:37.916777,3,"Salt, Product, Impurity1",LWE_concentration_based,LumpedRateModelWithPores,StericMassAction,True,True


In [25]:

# Load specific experiments
loaded = storage.load_results_by_selection(
    selections=[(set_id, exp.name) for exp in result.experiments],
    include_chromatogram=True,
)

loaded

[LoadedExperiment(experiment_set_id='f44d9a168cc9', experiment_set_name='my_study', experiment_name='experiment_1', result=SimulationResultWrapper(experiment_name='experiment_1', success=True, time=array([0.0000000e+00, 1.0000000e+00, 2.0000000e+00, ..., 1.6492000e+04,
        1.6493000e+04, 1.6493361e+04], shape=(16498,)), solution={'Salt': {}, 'Product': {}, 'Impurity1': {}}, errors=[], warnings=[], runtime_seconds=5.092850208282471, cadet_result=<CADETProcess.simulationResults.SimulationResults object at 0x72913017fe50>), experiment_config=ExperimentConfig(name='experiment_1', parameters={'flow_rate_mL_min': 1, 'equilibration_cv': 0, 'load_cv': 5, 'wash_cv': 5, 'elution_cv': 20, 'strip_cv': 5, 'load_salt_mM': 50, 'wash_salt_mM': 50, 'gradient_start_mM': 50, 'gradient_end_mM': 500, 'strip_salt_mM': 500, 'ph': 7, 'component_2_load_concentration': 0.1, 'component_3_load_concentration': 0.2, 'component_1_name': 'Salt', 'component_2_name': 'Product', 'component_3_name': 'Impurity1'}, com

In [7]:
p = plotting.plot_chromatogram_overlay(results, normalized=True, width=1200, height=600)
p

:Overlay
   .Curve.Experiment_1_hyphen_minus_Salt      :Curve   [time]   (Salt)
   .Curve.Experiment_1_hyphen_minus_Product   :Curve   [time]   (Product)
   .Curve.Experiment_1_hyphen_minus_Impurity1 :Curve   [time]   (Impurity1)
   .Curve.Experiment_2_hyphen_minus_Salt      :Curve   [time]   (Salt)
   .Curve.Experiment_2_hyphen_minus_Product   :Curve   [time]   (Product)
   .Curve.Experiment_2_hyphen_minus_Impurity1 :Curve   [time]   (Impurity1)
   .Curve.Experiment_3_hyphen_minus_Salt      :Curve   [time]   (Salt)
   .Curve.Experiment_3_hyphen_minus_Product   :Curve   [time]   (Product)
   .Curve.Experiment_3_hyphen_minus_Impurity1 :Curve   [time]   (Impurity1)